In [1]:
from datetime import datetime

from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.utils.operators import QubitPauliOperator
from pytket.partition import measurement_reduction,MeasurementBitMap, MeasurementSetup, PauliPartitionStrat
from pytket.backends.backendresult import BackendResult

from pytket.utils.operators import QubitPauliOperator
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

from scipy.optimize import minimize
from numpy import ndarray
from numpy.random import random_sample
from sympy import Symbol

import qnexus as qnx
from qnexus.references import CircuitRef


from nexus_dataclasses.backend_config import BackendConfig, QuantinuumConfig



# Example VQE workflow using Nexus


VQE example adapted from https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_variational_experiment_with_batching.ipynb

## Set up the VQE components

In [2]:
# 1. Synthesise Symbolic State-Preparation Circuit (hardware efficient ansatz)

symbols = [Symbol(f"p{i}") for i in range(4)]
symbolic_circuit = Circuit(2)
symbolic_circuit.X(0)
symbolic_circuit.Ry(symbols[0], 0).Ry(symbols[1], 1)
symbolic_circuit.CX(0, 1)
symbolic_circuit.Ry(symbols[2], 0).Ry(symbols[3], 0)

#render_circuit_jupyter(symbolic_circuit)

[X q[0]; Ry(p1) q[1]; Ry(p0) q[0]; CX q[0], q[1]; Ry(p2) q[0]; Ry(p3) q[0]; ]

In [3]:
# 2. Define Hamiltonian 
# coefficients in the Hamiltonian are obtained from PhysRevX.6.031007

coeffs = [-0.4804, 0.3435, -0.4347, 0.5716, 0.0910, 0.0910]
term0 = {
    QubitPauliString(
        {
            Qubit(0): Pauli.I,
            Qubit(1): Pauli.I,
        }
    ): coeffs[0]
}
term1 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.I}): coeffs[1]}
term2 = {QubitPauliString({Qubit(0): Pauli.I, Qubit(1): Pauli.Z}): coeffs[2]}
term3 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.Z}): coeffs[3]}
term4 = {QubitPauliString({Qubit(0): Pauli.X, Qubit(1): Pauli.X}): coeffs[4]}
term5 = {QubitPauliString({Qubit(0): Pauli.Y, Qubit(1): Pauli.Y}): coeffs[5]}
term_sum = {}
term_sum.update(term0)
term_sum.update(term1)
term_sum.update(term2)
term_sum.update(term3)
term_sum.update(term4)
term_sum.update(term5)
hamiltonian = QubitPauliOperator(term_sum)



In [4]:
# 3 Computing Expectation Values

# Computing Expectation Values for Pauli-Strings
def compute_expectation_paulistring(
    distribution: dict[tuple[int, ...], float], bitmap: MeasurementBitMap
) -> float:
    value = 0
    for bitstring, probability in distribution.items():
        value += probability * (sum(bitstring[i] for i in bitmap.bits) % 2)
    return ((-1) ** bitmap.invert) * (-2 * value + 1)



# 3.2 Computing Expectation Values for sums of Pauli-strings multiplied by coefficients
def compute_expectation_value(
    results: list[BackendResult],
    measurement_setup: MeasurementSetup,
    operator: QubitPauliOperator,
) -> float:
    energy = 0
    for pauli_string, bitmaps in measurement_setup.results.items():
        string_coeff = operator.get(pauli_string, 0.0)
        if string_coeff > 0:
            for bm in bitmaps:
                index = bm.circ_index
                distribution = results[index].get_distribution()
                value = compute_expectation_paulistring(distribution, bm)
                energy += complex(value * string_coeff).real
    return energy

In [5]:
# 4. Building our Objective function

class Objective:
    def __init__(
        self,
        symbolic_circuit: CircuitRef,
        problem_hamiltonian: QubitPauliOperator,
        n_shots_per_circuit: int,
        target: BackendConfig,
        iteration_number: int = 0,
        n_iterations: int = 10,
    ) -> None:
        r"""Returns the objective function needed for a variational
        procedure.
        """
        terms = [term for term in problem_hamiltonian._dict.keys()]
        self._symbolic_circuit: Circuit = symbolic_circuit.circuit
        self._hamiltonian: QubitPauliOperator = problem_hamiltonian
        self._nshots: int = n_shots_per_circuit
        self._measurement_setup: MeasurementSetup = measurement_reduction(
            terms, strat=PauliPartitionStrat.CommutingSets
        )
        self._iteration_number: int = iteration_number
        self._niters: int = n_iterations
        self._target = target


    def __call__(self, parameter: ndarray) -> float:
        value = self._objective_function(parameter)
        self._iteration_number += 1
        if self._iteration_number >= self._niters:
            self._iteration_number = 0
        return value
    
    def _objective_function(
        self,
        parameters: ndarray,
    ) -> float:
        assert len(parameters) == len(self._symbolic_circuit.free_symbols())
        circuit_list = self._build_circuits(parameters)
        
        # Execute circuits with Nexus
        execute_job_ref = qnx.execute(
            name=f"execute_job_VQE_{datetime.now()}_{self._iteration_number}",
            circuits=circuit_list,
            n_shots=[self._nshots]*len(circuit_list),
            target=self._target,
        )
        
        qnx.job.wait_for(execute_job_ref) # TODO assert success
        results = [item.get_backend_result() for item in qnx.job.execute.results(execute_job_ref)]
        
        expval = compute_expectation_value(
            results, self._measurement_setup, self._hamiltonian
        )
        return expval
    
    def _build_circuits(self, parameters: ndarray) -> list[CircuitRef]:
        circuit = self._symbolic_circuit.copy()
        symbol_dict = {s: p for s, p in zip(self._symbolic_circuit.free_symbols(), parameters)}
        circuit.symbol_substitution(symbol_dict)

        properties = {str(sym): val for sym, val in symbol_dict.items()} | {"iteration": self._iteration_number}

        with qnx.context.using_properties(**properties):
        
            # Upload the numerical state-prep circuit to Nexus
            qnx.circuit.upload(
                circuit=circuit,
                name=f"state prep circuit {self._iteration_number}",
            )
            circuit_list = []
            for mc in self._measurement_setup.measurement_circs:
                c = circuit.copy()
                c.append(mc)
                # Upload each measurement circuit to Nexus with correct params
                measurement_circuit_ref = qnx.circuit.upload(
                    circuit=c, 
                    name=f"state prep circuit {self._iteration_number}",
                )
                circuit_list.append(measurement_circuit_ref)
                
            # Compile circuits with Nexus
            compile_job_ref = qnx.compile(
                name=f"compile_job_VQE_{datetime.now()}_{self._iteration_number}",
                circuits=circuit_list,
                optimisation_level=2,
                target=self._target,
            )
            qnx.job.wait_for(compile_job_ref)
            compile_job_refs = qnx.job.compile.results(compile_job_ref)
            compiled_circuit_refs = [item.get_compiled_circuit() for item in compile_job_refs]

            for ref in compiled_circuit_refs:
                # Update with properties from context
                qnx.circuit.update(ref)
        return compiled_circuit_refs



## Set up the Nexus Project and run the VQE

In [17]:
# set up the project
project_ref = qnx.project.create(
    name=f"VQE_example_{str(datetime.now())}",
    description="A VQE done with qnexus",
)

# set this in the context
qnx.context.set_active_project(project_ref)

<Token var=<ContextVar name='qnexus_project' default=None at 0x17d19d030> at 0x17d5a5900>

In [18]:
# Set up the properties
qnx.project.add_property(
    name="iteration", 
    property_type="int", 
    description="The iteration number in my dihydrogen VQE experiment", 
)

for sym in symbolic_circuit.free_symbols():
    qnx.project.add_property(
        name=str(sym), 
        property_type="float",
        description=f"Our VQE {str(sym)} parameter", 
    )

In [19]:
# Upload our ansatz circuit

ansatz_ref = qnx.circuit.upload(
    circuit=symbolic_circuit,
    name="ansatz_circuit",
    description="The ansatz state-prep circuit for my dihydrogen VQE",
)

In [20]:
objective = Objective(
    symbolic_circuit = ansatz_ref,
    problem_hamiltonian = hamiltonian,
    n_shots_per_circuit = 500,
    n_iterations= 4,
    target = QuantinuumConfig(device_name="H1-1LE")
)

In [21]:
initial_parameters = random_sample(len(symbolic_circuit.free_symbols()))

result = minimize(
    objective,
    initial_parameters,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print(result.fun)
print(result.x)

-0.3774016000000001

[0.82083967 1.00659455 0.55822198 0.07807808]
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    4   F =-3.774016E-01    MAXCV = 0.000000E+00
   X = 8.208397E-01   1.006595E+00   5.582220E-01   7.807808E-02


# Use Nexus to Rescue a VQE workflow

For instance, lets say that some failure happened on the 2nd iteration (e.g. laptop ran out of battery) and we want to resume ASAP.

In the above we ran for 4 iterations, lets pretend that we actually wanted to run for 7 and it failed on the 4th one.

N.B. Assuming the minimizer has no internal state (or has been pickled).

In [6]:
# Get the project
project_ref = qnx.project.get_only(name="VQE_example_2024-04-24 11:17:07.035115")

# set this in the context
qnx.context.set_active_project(project_ref)


project_ref.df()


,name,description,id
0,VQE_example_2024-04-24 11:17:07.035115,A VQE done with qnexus,ee9464fc-8e50-4394-8c05-d05d8974a464


In [7]:
# Get the symbolic circuit
symbolic_circuit_ref = qnx.circuit.get_only(name="ansatz_circuit")

In [8]:
most_recent_circuits = qnx.circuit.get(name="final", project_ref=project_ref)

most_recent_circuits.summarize()

,resource,total_count
0,Circuit,8


In [9]:
most_recent_circuits_refs = most_recent_circuits.list()

most_recent_circuits_refs.df()

,name,description,iteration,p0,p1,p2,p3,project,id
0,state prep circuit 0-QuantinuumBackend-final,None,0,0.006595,0.078078,0.82084,0.558222,VQE_example_2024-04-24 11:17:07.035115,a1a47d1d-ce3c-41e3-b067-d26b85954d4d
1,state prep circuit 0-QuantinuumBackend-final,None,0,0.006595,0.078078,0.82084,0.558222,VQE_example_2024-04-24 11:17:07.035115,59593dc6-203e-4a37-9d6d-77183e9ca57a
2,state prep circuit 1-QuantinuumBackend-final,None,1,0.006595,0.078078,1.82084,0.558222,VQE_example_2024-04-24 11:17:07.035115,a60f4d8a-d426-41f2-99c9-b9c639b7dd7c
3,state prep circuit 1-QuantinuumBackend-final,None,1,0.006595,0.078078,1.82084,0.558222,VQE_example_2024-04-24 11:17:07.035115,75324875-2d7a-4271-b5f3-abaaab133f6e
4,state prep circuit 2-QuantinuumBackend-final,None,2,1.006595,0.078078,0.82084,0.558222,VQE_example_2024-04-24 11:17:07.035115,39d601de-79f9-430f-be04-0e6dbbfd12d7
5,state prep circuit 2-QuantinuumBackend-final,None,2,1.006595,0.078078,0.82084,0.558222,VQE_example_2024-04-24 11:17:07.035115,b761406a-e559-47db-80c6-f27866a334f4
6,state prep circuit 3-QuantinuumBackend-final,None,3,1.006595,0.078078,0.82084,1.558222,VQE_example_2024-04-24 11:17:07.035115,fd199f36-2763-4990-bccb-254808ca56a5
7,state prep circuit 3-QuantinuumBackend-final,None,3,1.006595,0.078078,0.82084,1.558222,VQE_example_2024-04-24 11:17:07.035115,1846666e-0e18-4bf6-83f8-08bf775839c7


In [10]:
# Get the latest circuit to get the new 'initial_parameters'
latest_circuit: CircuitRef = most_recent_circuits_refs[-1]

latest_circuit_properties = latest_circuit.annotations.properties

latest_circuit.df()

,name,description,iteration,p0,p1,p2,p3,project,id
0,state prep circuit 3-QuantinuumBackend-final,None,3,1.006595,0.078078,0.82084,1.558222,VQE_example_2024-04-24 11:17:07.035115,1846666e-0e18-4bf6-83f8-08bf775839c7


In [11]:
# Get what iteration we were on (from the latest circuit)

last_iteration_count = latest_circuit_properties.pop("iteration")

print(last_iteration_count)

# Retreive the params and check them
new_starting_params = list(latest_circuit_properties.values())
print(new_starting_params)


3
[1.0065945386886597, 0.07807808369398117, 0.8208396434783936, 1.5582219362258911]


In [12]:
# Build the Objective and run 'minimize' to continue the experiment
objective = Objective(
    symbolic_circuit_ref,
    hamiltonian,
    n_shots_per_circuit = 500,
    iteration_number=last_iteration_count, # resume from 3rd iteration of 7
    n_iterations = 7,
    target = QuantinuumConfig(device_name="H1-1LE")
)

result = minimize(
    objective,
    new_starting_params,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print(result.fun)
print(result.x)

-0.48302120000000004
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    7   F =-4.830212E-01    MAXCV = 0.000000E+00
   X = 1.462614E+00  -7.616175E-02   6.884107E-01   2.585031E+00

[ 1.46261419 -0.07616175  0.68841072  2.58503116]
